In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cross_validation import train_test_split
from sklearn import metrics

#for displaying result
import csv
from IPython.display import HTML, display
import tabulate

#these two lines changes jupyter's variable display to put each variable on its own line
#that way, we can dump mulitple variables from a single code cell (without them overwriting the previous)
#from IPython.core.interactiveshell import InteractiveShell
#InteractiveShell.ast_node_interactivity = "all"

/Users/dipierre/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Data Files

In [2]:
def loadAllDataFiles():
    filenames=[]
    filenames.append("./data/FinAid_Labeled.csv")
    dumpColumnTitles()
    for file in filenames:
        X_train, X_test, y_train, y_test = loadOneFile(file)
        runAllVectorizers(X_train, X_test, y_train, y_test)


def loadOneFile(filename):
    df = pd.read_csv(filename)

    X = df.question
    y = df.Intent_Number
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
    
    return X_train, X_test, y_train, y_test


def dumpColumnTitles():
    f.write("Stop Words, nGram Range, Max Doc Frequency, Min Doc Frequency, ")
    for model in models:
        f.write(model['name'] + ', ')
    f.write ('\n')

##  Vectorizers

In [28]:
def runAllVectorizers(X_train, X_test, y_train, y_test):
    i = 0
    for vect in vectorizers:
        i += 1
        print ("Running Vectorizer {} of {}".format(i, len(vectorizers)), end="\r")
        X_train_dtm, X_test_dtm = runOneVectorizer(vect,X_train, X_test, y_train, y_test)
        runAllModels(X_train_dtm, X_test_dtm, y_train, y_test)
        f.write('\n')
    print("                                                         ", end="\r")
    

def runOneVectorizer(vect,X_train, X_test, y_train, y_test):
    # learn training data vocabulary, then use it to create a document-term matrix
    X_train_dtm = vect.fit_transform(X_train)

    # transform testing data (using fitted vocabulary) into a document-term matrix
    X_test_dtm = vect.transform(X_test)

    if vect.stop_words is None:
        f.write("None")
    else:
        f.write(str(vect.stop_words))
    f.write(',')
    f.write(str(vect.ngram_range).replace(',','-'))
    f.write(',')
    f.write(str(vect.max_df))
    f.write(',')
    f.write(str(vect.min_df))
    f.write(',')
    
    return X_train_dtm, X_test_dtm


## Models

In [8]:
def runAllModels(X_train_dtm, X_test_dtm, y_train, y_test):
    for model in models:
        runOneModel(model['model'],X_train_dtm, X_test_dtm, y_train, y_test)


def runOneModel(model,X_train_dtm, X_test_dtm, y_train, y_test):
    # train the model using X_train_dtm (timing it with an IPython "magic command")
    #%time model.fit(X_train_dtm, y_train)
    model.fit(X_train_dtm, y_train)

    # make class predictions for X_test_dtm
    y_pred_class = model.predict(X_test_dtm)

    # calculate accuracy of class predictions
    f.write(str(metrics.accuracy_score(y_test, y_pred_class)))
    f.write(',')    
    

## Initialize Vectorizers & Models, then Run

In [9]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier

#tweak the vectorizing settings here:
vectorizers=[]
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 1), min_df=1, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 1), min_df=1, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 2), min_df=1, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=1, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 1), min_df=2, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 1), min_df=2, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 2), min_df=2, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2, max_df=1.0))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 1), min_df=1, max_df= 30))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 1), min_df=1, max_df= 30))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 2), min_df=1, max_df= 30))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=1, max_df= 30))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 1), min_df=2, max_df= 30))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 1), min_df=2, max_df= 30))
vectorizers.append(CountVectorizer(stop_words= None,     ngram_range=(1, 2), min_df=2, max_df= 30))
vectorizers.append(CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2, max_df= 30))

models = []
models.append({'model':MultinomialNB(), 'name': 'Naive Bayes'})
models.append({'model':LogisticRegression(), 'name': 'Logistic Regression'})
models.append({'model':svm.SVC(), 'name': 'Linear SVC'})
models.append({'model':RandomForestClassifier(n_estimators = 50), 'name': 'Random Forest'})

In [26]:
filename='output.csv'
f = open(filename, 'w')
loadAllDataFiles()
f.close()

with open(filename, 'r') as f:
    data = list(csv.reader(f))

display(HTML(tabulate.tabulate(data, tablefmt='html')))


Stop Words,nGram Range,Max Doc Frequency,Min Doc Frequency,Naive Bayes,Logistic Regression,Linear SVC,Random Forest,
None,(1- 1),1.0,1,0.274193548387,0.39247311828,0.161290322581,0.440860215054,
english,(1- 1),1.0,1,0.306451612903,0.376344086022,0.161290322581,0.435483870968,
None,(1- 2),1.0,1,0.306451612903,0.387096774194,0.161290322581,0.403225806452,
english,(1- 2),1.0,1,0.301075268817,0.376344086022,0.161290322581,0.397849462366,
None,(1- 1),1.0,2,0.290322580645,0.39247311828,0.215053763441,0.408602150538,
english,(1- 1),1.0,2,0.322580645161,0.365591397849,0.225806451613,0.403225806452,
None,(1- 2),1.0,2,0.301075268817,0.39247311828,0.177419354839,0.39247311828,
english,(1- 2),1.0,2,0.322580645161,0.354838709677,0.220430107527,0.39247311828,
None,(1- 1),30,1,0.290322580645,0.338709677419,0.161290322581,0.344086021505,
english,(1- 1),30,1,0.290322580645,0.311827956989,0.161290322581,0.376344086022,
